In [37]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import math
import glob
import os
from IPython.display import HTML

In [2]:

class Dataset():
    def __init__(self, path, fl, cx, cy):
        self.path = path
        wildcard = os.path.join(self.path, '*.jpg')
        self.image_paths = sorted([f for f in glob.glob(wildcard)])
        self.images = [self.read(f) for f in self.image_paths]
        self.image_count = len(self.image_paths)
        self.camera_matrix = np.array([[fl, 0.0, cx],
                                       [0.0, fl, cy],
                                       [0.0, 0.0, 1.0]])
        
    def read(self, img):
        image = cv2.imread(img)
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        

In [3]:
path = '../../boulder/gopro/data/dash2/images'
fl = 873.187 # Focal Length
cx = 953.285 # Principal Point X
cy = 731.167 # Pricipal Point Y

dataset = Dataset(path, fl, cx, cy)
print('Dataset Images', dataset.image_count)

Dataset Images 108


In [5]:

position_figure = plt.figure(figsize=(10,10))
position_axes = position_figure.add_subplot(1, 1, 1)

# error_figure = plt.figure()
# rotation_error_axes = error_figure.add_subplot(1, 1, 1)
# rotation_error_list = []
# frame_index_list = []

position_axes.set_aspect('equal', adjustable='box')

In [41]:
# Much taken from: https://github.com/yoshimasa1700/mono_vo_python/blob/master/main.py
feature_detector = cv2.ORB_create()
#feature_detector = cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)
lk_params = dict(winSize=(21, 21),criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03))

current_pos = np.zeros((3, 1))
current_rot = np.eye(3)

prev_image = None

camera_matrix = dataset.camera_matrix

ims = []
positions = []
   
nframes = 20
start = 10
for index in range(start, start+nframes):
    image = dataset.images[index]
    keypoint, des = feature_detector.detectAndCompute(image, None)
    if prev_image is None:
        prev_image = image
        prev_keypoint = keypoint
        prev_des = des
        continue

    points = np.array(list(map(lambda x: [x.pt], prev_keypoint)),dtype=np.float32)
    p1, st, err = cv2.calcOpticalFlowPyrLK(prev_image,
                                           image, points,
                                           None, **lk_params)
    E, mask = cv2.findEssentialMat(p1, points, camera_matrix,cv2.RANSAC, 0.999, 1.0, None)
    points, R, t, mask = cv2.recoverPose(E, p1, points, camera_matrix)
    scale = 1.0

    current_pos += current_rot.dot(t) * scale
    current_rot = R.dot(current_rot)



    positions.append((current_pos[0][0], current_pos[2][0]))
    img = cv2.drawKeypoints(image, keypoint, None)
    ims.append(img)
    
    prev_image = image
    prev_keypoint = keypoint

In [42]:
x,y = zip(*positions)
x0 = list(np.zeros(len(positions)))
y0 = list(range(len(positions)))

In [43]:
import numpy as np
from celluloid import Camera
from matplotlib import gridspec


fig = plt.figure(figsize=(10, 6)) 
gs = gridspec.GridSpec(1, 2, width_ratios=[4, 1]) 
ax1 = plt.subplot(gs[0])
ax1.axis('off')
ax2 = plt.subplot(gs[1])
ax2.scatter(x0, y0, c='b')

plt.tight_layout()

camera = Camera(fig)

plt.xlim(-3, 3)
plt.ylim(-.5, 20)

for i in range(len(positions)):
    ax2.scatter(x0, y0, c='b')
    ax2.scatter(x[:i], y[:i], c='r')
    ax1.imshow(ims[i])
    camera.snap()
    
anim = camera.animate(blit=True)
    
anim.save('animation.mp4', fps=1, extra_args=['-vcodec', 'libx264'])
plt.close(anim._fig)
HTML(anim.to_html5_video())